In [42]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import json

In [43]:
# Load the Excel file
file_path = 'DataDownload.xlsx'
xls = pd.ExcelFile(file_path)

In [44]:
# Create a single dataframe with an additional 'Year' column
df_list = []
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    df['Year'] = sheet
    df_list.append(df)

# Concatenate all dataframes into one
combined_df = pd.concat(df_list, ignore_index=True)

# Strip whitespace and remove multiple spaces in both dataframes
combined_df['COUNTRY'] = combined_df['COUNTRY'].str.strip().replace(r'\s+', ' ', regex=True).replace(r'\*', '', regex=True)
combined_df['POST COUNTRY'] = combined_df['POST COUNTRY'].str.strip().replace(r'\s+', ' ', regex=True).replace(r'\*', '', regex=True)

# Replace 'Post City' values when 'Post City' is 'Belgium' with 'Brussels'
combined_df.loc[combined_df['POST CITY'] == 'Belgium', 'POST CITY'] = 'Brussels'

# Replace 'Post Country' values when 'Post Country' is 'Brussels' with 'Belgium'
combined_df.loc[combined_df['POST COUNTRY'] == 'Brussels', 'POST COUNTRY'] = 'Belgium'

# Define a dictionary for replacements
replacements = {
    "Korea Republic of (South Korea)": "South Korea",
    "Russian Federation": "Russia",
    "Slovak Republic": "Slovakia",
    "Czechia": "Czech Republic",
    "Lao People's Democratic Republic": "Laos",
    "Korea Democratic People's Republic of (North Korea)": "North Korea",
    "Brunei Darussalam": "Brunei",
    "Türkiye": "Turkey",
    "Cura�ao": "Curacao",
    "Dominica Republic": "Dominican Republic",
    "Bolivia Plurinational State of": "Bolivia",
    "Iran Islamic Republic of": "Iran",
    "Moldova Republic of": "Moldova",
    "Tanzania United Republic of": "Tanzania",
    "Vatican City State": "Holy See",
    "Holy See (Vatican City State)": "Holy See",
    "Vatican City": "Holy See",
    "Venezuela Bolivarian Republic of": "Venezuela",
    "Macedonia the former Yugoslav Republic of": "North Macedonia",
    "Palestine": "State of Palestine",
    "Palestinian Territories": "State of Palestine",
    "Portual": "Portugal",
    "New Caledonia (France)": "New Caledonia",
    "New Caledonia (French Territory)": "New Caledonia",
    "New Caledonia (French territory)": "New Caledonia",
    "Democratic the Congo": "DR Congo",
    "Democratic The Congo": "DR Congo",
    "Democratic Republic of The Congo": "DR Congo",
    "Nigaragua": "Nicaragua",
    "Cote D'Ivoire": "Cote d'Ivoire",
    "Sint Maarten (Dutch territory)": "Sint Maarten",
    "Swaziland": "Eswatini",
    "European External Action Service": "European Union",
    "Bermuda (British overseas territory)": "Bermuda",
    "Guam (US territory)": "Guam",
    "Micronesia Federated States of": "Micronesia, Federated States of",
    "Cape Verde": "Cabo Verde",
    "Syria": "Syrian Arab Republic"
    
}

combined_df['COUNTRY'] = combined_df['COUNTRY'].replace(replacements)
combined_df['POST COUNTRY'] = combined_df['POST COUNTRY'].replace(replacements)

# Drop duplicates to ensure unique counts per year
unique_df = combined_df.drop_duplicates(subset=['COUNTRY', 'Year'])

# Count unique occurrences of each value in the 'COUNTRY' column over all years
country_unique_counts = unique_df['COUNTRY'].value_counts()

# Drop duplicates to ensure unique counts per year for 'POST COUNTRY'
unique_post_df = combined_df.drop_duplicates(subset=['POST COUNTRY', 'Year'])

# Count unique occurrences of each value in the 'POST COUNTRY' column over all years
post_country_unique_counts = unique_post_df['POST COUNTRY'].value_counts()

# Present the results in tables
country_unique_counts_df = country_unique_counts.reset_index()
country_unique_counts_df.columns = ['Country', 'Count']

post_country_unique_counts_df = post_country_unique_counts.reset_index()
post_country_unique_counts_df.columns = ['Post Country', 'Count']

# Filter dataframes to show only countries with counts less than 5
country_less_than_5 = country_unique_counts_df[country_unique_counts_df['Count'] < 5]
post_country_less_than_5 = post_country_unique_counts_df[post_country_unique_counts_df['Count'] < 5]

#country_unique_counts_df, post_country_unique_counts_df


In [45]:
post_counts = combined_df.groupby(['COUNTRY', 'POST COUNTRY', 'Year']).size().reset_index(name='POST COUNT')
top_post_counts = post_counts.sort_values(by='POST COUNT', ascending=False).head(50)
top_post_counts
pd.set_option('display.max_rows', 1000)
#post_counts.loc[(post_counts["COUNTRY"] == "United States") & (post_counts["Year"] == "2016")]
#post_counts.head(20)

In [46]:
# Select the unique values for each country/year combination
node_attributes = combined_df[['COUNTRY', 'Year', 'POPULATION (M)', 'GDP (B, USD)', 'G20 RANK', 'OECD RANK', 'ASIA RANK', 'OVERALL RANK']].drop_duplicates()

# Reset the index for the new dataframe
node_attributes.reset_index(drop=True, inplace=True)

#node_attributes

In [47]:
# Load and inspect the JSON file 
with open('network-power.json', 'r') as file:
    json_data = json.load(file)

json_data.keys()

dict_keys(['global', 'data', 'countries'])

In [48]:
# Convert each list under the main keys into separate DataFrames
df_global = pd.DataFrame(json_data['global'])
df_data = pd.DataFrame(json_data['data'])
df_countries = pd.DataFrame(json_data['countries'])

# Show a brief overview of each DataFrame
df_global.head() 
df_data.head()
#df_countries.head()
# Update the value in df_global_simplified
df_global.loc[df_global['name'] == 'CÃ´te d\'Ivoire', 'name'] = 'Cote d\'Ivoire'

In [49]:
# Extracting 'menu' items for 'Economy'
economy_menu_items = df_data[df_data['name'] == 'Economy']['menu'].values[0]
culture_menu_items = df_data[df_data['name'] == 'Culture']['menu'].values[0]
defence_menu_items = df_data[df_data['name'] == 'Defence']['menu'].values[0]
diplomacy_menu_items = df_data[df_data['name'] == 'Diplomacy']['menu'].values[0]

In [50]:
# Parsing 'menu' items to find 'Trade' and 'Investment' data
trade_data = []
investment_data = []

for item in economy_menu_items:
    if item['name'] == 'Trade':
        trade_data = [
            {
                "countryA": data_item["countryA"],
                "countryB": data_item["countryB"],
                # normValue is the percentage of country A's trade that is with country B. 
                # For example, 26.9% of Papua New Guinea's trade is with Australia (2.83 BN is flowing FROM PG to AU)
                #                 {
                #   "id": "i0",
                #   "year": 2023,
                #   "countryA": "PG",
                #   "countryB": "AU",
                #   "normValue": 0.2687219178,
                #   "rawValue": 2831083050
                # },
                    
                "bilateralTradeNormValue": data_item["normValue"],
                "bilateralTradeRawValue": data_item["rawValue"]
            }
            for data_item in item['data']
        ]
    elif item['name'] == 'Investment':
        investment_data = [
            {
                "countryB": data_item["countryA"],
                "countryA": data_item["countryB"],
                # Investment of foreign capital investments from country B to A (flip A to B direction for graph).
                # $10.1 BN is invested into Australia from Japan (that's 5.4% of all the investment into Australia)
                #             {
                #   "id": "i2",
                #   "year": 2023,
                #   "countryA": "AU",
                #   "countryB": "JP",
                #   "normValue": 0.0540781862,
                #   "rawValue": 10134585476
                # },
                "foreignCapitalInvestmentNormValue": data_item["normValue"],
                "foreignCapitalInvestmentRawValue": data_item["rawValue"]
            }
            for data_item in item['data']
        ]

# Convert lists to DataFrames
df_trade = pd.DataFrame(trade_data)
df_investment = pd.DataFrame(investment_data)

# Merging the DataFrames on 'countryA' and 'countryB'
df_combined = pd.merge(df_trade, df_investment, on=["countryA", "countryB"], how="outer")

# Parsing 'menu' items to find 'Culture' data
culture_online_search_data = []
culture_migrant_destinations_data = []
culture_travel_destinations_data = []
culture_foreign_media_flows_data = []

for item in culture_menu_items:
    if item['name'] == 'Online search interest':
        culture_online_search_data = [
            {
                "countryA": data_item["countryA"],
                "countryB": data_item["countryB"],
                # 3% of searches from AU to RU (out of 26 countries that are ranked)
                #                 {
                #   "year": 2023,
                #   "id": "i4",
                #   "countryA": "AU",
                #   "countryB": "RU",
                #   "rawValue": 0.0297251321
                # },
                "onlineSearchInterestRawValue": data_item["rawValue"]
            }
            for data_item in item['data']
        ]
    elif item['name'] == 'Migrant destinations':
        culture_migrant_destinations_data = [
            {
                "countryA": data_item["countryA"],
                "countryB": data_item["countryB"],
                # 3.1 % of migrants from Nepal settle in Australia, around 80K
                #                 {
                #   "year": 2023,
                #   "id": "i11",
                #   "countryA": "NP",
                #   "countryB": "AU",
                #   "rawValue": 79427,
                #   "normValue": 0.0305503267,
                #   "flipped": true
                # },
                "migrantDestinationsRawValue": data_item["rawValue"],
                "migrantDestinationsNormValue": data_item["normValue"]
            }
            for data_item in item['data']
        ]
    elif item['name'] == 'Travel destinations':
        culture_travel_destinations_data = [
            {
                "countryA": data_item["countryA"],
                "countryB": data_item["countryB"],
                # Raw number and percentage of travellers from country A to country B. 
                # 9.2% of travellers from Australia visited China in 2020 (176,393)
                #                 {
                #   "year": 2023,
                #   "id": "i12",
                #   "countryA": "AU",
                #   "countryB": "CN",
                #   "rawValue": 176393.055092997,
                #   "normValue": 0.0917605295
                # },
                "travelDestinationsRawValue": data_item["rawValue"],
                "travelDestinationsNormValue": data_item["normValue"]
            }
            for data_item in item['data']
        ]
    elif item['name'] == 'Foreign media flows':
        culture_foreign_media_flows_data = [
            {
                "countryA": data_item["countryA"],
                "countryB": data_item["countryB"],
                # 1.1 % of relevant internet searches in Australia target media from Japan
                #                 {
                #   "year": 2023,
                #   "id": "i16",
                #   "countryA": "AU",
                #   "countryB": "JP",
                #   "rawValue": 0.010934827
                # },
                "foreignMediaFlowsRawValue": data_item["rawValue"]
            }
            for data_item in item['data']
        ]

# Convert lists to DataFrames
df_online_search = pd.DataFrame(culture_online_search_data)
df_migrant_destinations = pd.DataFrame(culture_migrant_destinations_data)
df_travel_destinations = pd.DataFrame(culture_travel_destinations_data)
df_foreign_media_flows = pd.DataFrame(culture_foreign_media_flows_data)

# Merging the DataFrames on 'countryA' and 'countryB'
df_combined = df_combined.merge(df_online_search, on=["countryA", "countryB"], how="outer")
df_combined = df_combined.merge(df_migrant_destinations, on=["countryA", "countryB"], how="outer")
df_combined = df_combined.merge(df_travel_destinations, on=["countryA", "countryB"], how="outer")
df_combined = df_combined.merge(df_foreign_media_flows, on=["countryA", "countryB"], how="outer")

for item in defence_menu_items:
    if item['name'] == 'Arms trade':
        defence_arms_trade_data = [
            {
                "countryA": data_item["countryA"],
                "countryB": data_item["countryB"],
                # 0% of Australia's arms trade is with China ($2 M)
                #                 {
                #   "year": 2023,
                #   "id": "i6",
                #   "countryA": "AU",
                #   "countryB": "CN",
                #   "rawValue": 2000000,
                #   "normValue": 0.0001648533
                # },
                "armsTradeNormValue": data_item["normValue"],
                "armsTradeRawValue": data_item["rawValue"]
            }
            for data_item in item['data']
        ]
    elif item['name'] == 'Joint training exercises':
        defence_joint_training_data = [
            {
                "countryA": data_item["countryA"],
                "countryB": data_item["countryB"],
                # 4.9% of Australia's joint training exercises are with India (24 joint exercises)
                #                 {
                #   "year": 2023,
                #   "id": "i8",
                #   "countryA": "AU",
                #   "countryB": "IN",
                #   "rawValue": 24,
                #   "normValue": 0.0491803279
                # },
                "jointTrainingNormValue": data_item["normValue"],
                "jointTrainingRawValue": data_item["rawValue"]
            }
            for data_item in item['data']
        ]
    elif item['name'] == 'Defence Dialogues':
        defence_dialogues_data = [
            {
                "countryA": data_item["countryA"],
                "countryB": data_item["countryB"],
                # Australia held 4 defence dialogues with India in 2020-21 (10.3% of Australia's total)
                #                 {
                #   "year": 2023,
                #   "id": "i20",
                #   "countryA": "AU",
                #   "countryB": "IN",
                #   "rawValue": 4,
                #   "normValue": 0.1025641026,
                #   "comments": "3"
                # },
                "defenceDialoguesNormValue": data_item["normValue"],
                "defenceDialoguesRawValue": data_item["rawValue"]
            }
            for data_item in item['data']
        ]

# Convert lists to DataFrames
df_arms_trade = pd.DataFrame(defence_arms_trade_data)
df_joint_training = pd.DataFrame(defence_joint_training_data)
df_defence_dialogues = pd.DataFrame(defence_dialogues_data)

# Merging the DataFrames on 'countryA' and 'countryB'
df_combined = df_combined.merge(df_arms_trade, on=["countryA", "countryB"], how="outer")
df_combined = df_combined.merge(df_joint_training, on=["countryA", "countryB"], how="outer")
df_combined = df_combined.merge(df_defence_dialogues, on=["countryA", "countryB"], how="outer")

# Parsing 'menu' items to find 'Diplomacy' data
diplomacy_un_voting_data = []
diplomacy_dialogues_data = []

for item in diplomacy_menu_items:
    if item['name'] == 'UN voting alignment':
        diplomacy_un_voting_data = [
            {
                "countryA": data_item["countryA"],
                "countryB": data_item["countryB"],
                # Bangladesh is Australia's 14th most closely aligned country in UN voting (54%)
                # This isn't a very useful network measure - hard to interpret directionality 
                #               "data": [
                # {
                #   "year": 2023,
                #   "id": "i18",
                #   "countryA": "AU",
                #   "countryB": "BA",
                #   "rawValue": 14,
                #   "normValue": 0.5416666667
                # },
                "unVotingNormValue": data_item["normValue"],
                "unVotingRawValue": data_item["rawValue"]
            }
            for data_item in item['data']
        ]
    elif item['name'] == 'Diplomatic Dialogues':
        diplomacy_dialogues_data = [
            {
                "countryA": data_item["countryA"],
                "countryB": data_item["countryB"],
                # Australia help 6 diplomatic dialogues with Japan in 2021 (16.7% of Australia's total)
                #                 {
                #   "year": 2023,
                #   "id": "i22",
                #   "countryA": "AU",
                #   "countryB": "JP",
                #   "rawValue": 6,
                #   "normValue": 0.1666666667,
                #   "comments": "1"
                # },
                "diplomaticDialoguesNormValue": data_item["normValue"],
                "diplomaticDialoguesRawValue": data_item["rawValue"]
            }
            for data_item in item['data']
        ]

# Convert lists to DataFrames
df_un_voting = pd.DataFrame(diplomacy_un_voting_data)
df_diplomatic_dialogues = pd.DataFrame(diplomacy_dialogues_data)

# Merging the DataFrames on 'countryA' and 'countryB'
# df_combined = df_combined.merge(df_un_voting, on=["countryA", "countryB"], how="outer")
df_combined = df_combined.merge(df_diplomatic_dialogues, on=["countryA", "countryB"], how="outer")

In [51]:
# Create a simplified DataFrame from 'df_global' with only 'id' and 'name' columns
df_global_simplified = df_global[['id', 'name']]

# Display the new DataFrame
df_global_simplified

# Step 1: Extract unique country and post country names from post_counts
unique_countries = set(post_counts['COUNTRY']).union(set(post_counts['POST COUNTRY']))
unique_countries_2 = set(df_global_simplified['name'])

# Step 2: Create a DataFrame with these unique country names
country_df = pd.DataFrame(unique_countries, columns=['name'])

merged_df = country_df.merge(df_global_simplified, on='name', how='outer')


# Define a dictionary for the country codes to replace NaN values
replacement_dict = {
    'Abkhazia': 'AK',
    'Aruba': 'AB',
    'Curacao': 'CC',
    'French Guiana': 'FG',
    'Jerusalem': 'JE',
    'Niue': 'NU',
    'Northern Cyprus': 'NCY',
    'Puerto Rico (US territory)': 'PR',
    'Somaliland': 'SOM',
    'South Ossetia': 'OS'
}

# Replace NaN values in the 'id' column based on the 'name' column
merged_df['id'] = merged_df.apply(
    lambda row: replacement_dict.get(row['name'], row['id']), axis=1)

merged_df_sorted = merged_df.sort_values(by='id').reset_index(drop=True)

In [52]:
# Create a mapping dictionary from merged_df_sorted
name_to_id_map = pd.Series(merged_df_sorted.id.values, index=merged_df_sorted.name).to_dict()

# merged_df_sorted.id.values: This part accesses the 'id' column of the dataframe merged_df_sorted and then uses .values to get these column values 
# as an array.

# merged_df_sorted.name: Here, the 'name' column of the dataframe merged_df_sorted is accessed. This will be used as the index in the next step.

# pd.Series(...): This function is used to create a pandas Series object. The first argument, merged_df_sorted.id.values, provides the data for 
# the Series, and the second argument, index=merged_df_sorted.name, sets the index of the Series to the values of the 'name' column.

# .to_dict(): This method converts the Series object into a dictionary. The index values (names) from the Series become the keys of the dictionary, 
# and the data values (IDs) become the values of the dictionary.

# Replace COUNTRY and POST COUNTRY values with their corresponding id values
post_counts['COUNTRY'] = post_counts['COUNTRY'].map(name_to_id_map)
post_counts['POST COUNTRY'] = post_counts['POST COUNTRY'].map(name_to_id_map)
node_attributes['COUNTRY'] = node_attributes['COUNTRY'].map(name_to_id_map)

In [53]:
countries_df = merged_df_sorted

# Updated UN Geoscheme with additional countries and regions
un_geoscheme = {
    "Northern Africa": ["Algeria", "Egypt", "Libya", "Morocco", "Sudan", "Tunisia", "Western Sahara"],
    "Eastern Africa": ["British Indian Ocean Territory", "Burundi", "Comoros", "Djibouti", "Eritrea", "Ethiopia", "French Southern Territories", "Kenya", "Madagascar", "Malawi", "Mauritius", "Mayotte", "Mozambique", "Réunion", "Rwanda", "Seychelles", "Somalia", "Somaliland", "South Sudan", "Uganda", "Tanzania", "Zambia", "Zimbabwe"],
    "Middle Africa": ["Angola", "Cameroon", "Central African Republic", "Chad", "Congo", "DR Congo", "Equatorial Guinea", "Gabon", "Sao Tome and Principe"],
    "Southern Africa": ["Botswana", "Eswatini", "Lesotho", "Namibia", "South Africa"],
    "Western Africa": ["Benin", "Burkina Faso", "Cabo Verde", "Cote d'Ivoire", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Liberia", "Mali", "Mauritania", "Niger", "Nigeria", "Saint Helena", "Senegal", "Sierra Leone", "Togo"],
    "Caribbean": ["Anguilla", "Antigua and Barbuda", "Aruba", "Bahamas", "Barbados", "Bonaire", "Sint Eustatius and Saba", "British Virgin Islands", "Cayman Islands", "Cuba", "Curacao", "Dominica", "Dominican Republic", "Grenada", "Guadeloupe", "Haiti", "Jamaica", "Martinique", "Montserrat", "Puerto Rico (US territory)", "Saint Barthelemy", "Saint Kitts and Nevis", "Saint Lucia", "Dutch Antilles", "Sint Maarten", "Saint Vincent and the Grenadines", "Trinidad and Tobago", "Turks and Caicos Islands", "United States Virgin Islands"],
    "Central America": ["Belize", "Costa Rica", "El Salvador", "Guatemala", "Honduras", "Mexico", "Nicaragua", "Panama"],
    "South America": ["Argentina", "Bolivia", "Bouvet Island", "Brazil", "Chile", "Colombia", "Ecuador", "Falkland Islands (Malvinas)", "French Guiana", "Guyana", "Paraguay", "Peru", "South Georgia and the South Sandwich Islands", "Suriname", "Uruguay", "Venezuela"],
    "Northern America": ["Bermuda", "Canada", "Greenland", "Saint Pierre and Miquelon", "United States"],
    "Central Asia": ["Kazakhstan", "Kyrgyzstan", "Tajikistan", "Turkmenistan", "Uzbekistan"],
    "Eastern Asia": ["China", "China - Hong Kong Special Administrative Region", "China - Macao Special Administrative Region", "North Korea", "Japan", "Mongolia", "South Korea", "Taiwan"],
    "Southeastern Asia": ["Brunei", "Cambodia", "Indonesia", "Laos", "Malaysia", "Myanmar", "Philippines", "Singapore", "Thailand", "Timor-Leste", "Vietnam"],
    "Southern Asia": ["Afghanistan", "Bangladesh", "Bhutan", "India", "Iran", "Maldives", "Nepal", "Pakistan", "Sri Lanka"],
    "Western Asia": ["Armenia", "Azerbaijan", "Bahrain", "Cyprus", "Northern Cyprus", "Georgia", "South Ossetia", "Abkhazia", "Iraq", "Israel", "Jordan", "Kuwait", "Lebanon", "Oman", "Qatar", "Saudi Arabia", "State of Palestine", "Syrian Arab Republic", "Turkey", "United Arab Emirates", "Yemen", "Jerusalem"],
    "Eastern Europe": ["Belarus", "Bulgaria", "Czech Republic", "Hungary", "Poland", "Moldova", "Romania", "Russia", "Slovakia", "Ukraine"],
    "Northern Europe": ["Åland Islands", "Channel Islands (Guernsey, Jersey, Sark)", "Denmark", "Estonia", "Faroe Islands", "Finland", "Iceland", "Ireland", "Isle of Man", "Latvia", "Lithuania", "Norway", "Svalbard and Jan Mayen Islands", "Sweden", "United Kingdom"],
    "Southern Europe": ["Albania", "Andorra", "Bosnia and Herzegovina", "Croatia", "Gibraltar", "Greece", "Holy See", "Italy", "Malta", "Montenegro", "Portugal", "San Marino", "Serbia", "Slovenia", "Spain", "North Macedonia", "Kosovo"],
    "Western Europe": ["Austria", "Belgium", "France", "Germany", "Liechtenstein", "Luxembourg", "Monaco", "Netherlands", "Switzerland"],
    "Australia and New Zealand": ["Australia", "Christmas Island", "Cocos (Keeling) Islands", "Heard Island and McDonald Islands", "New Zealand", "Norfolk Island"],
    "Melanesia": ["Fiji", "New Caledonia", "Papua New Guinea", "Solomon Islands", "Vanuatu"],
    "Micronesia": ["Guam", "Kiribati", "Marshall Islands", "Micronesia, Federated States of", "Nauru", "Northern Mariana Islands", "Palau", "United States Minor Outlying Islands"],
    "Polynesia": ["Samoa", "Tuvalu", "Wallis and Futuna Islands", "Niue", "French Polynesia", "Tonga", "American Samoa", "Cook Islands", "Tokelau"],
    "European Union": ["European Union"],
    "Antarctica": ["Antarctica"]
}

# Region mapping
region_mapping = {
    "Africa": ["Northern Africa", "Eastern Africa", "Middle Africa", "Southern Africa", "Western Africa"],
    "Americas": ["Caribbean", "Central America", "South America", "Northern America"],
    "Asia": ["Eastern Asia", "Central Asia", "Southern Asia", "Western Asia", "Southeastern Asia"],
    "Europe": ["European Union", "Eastern Europe", "Northern Europe", "Southern Europe", "Western Europe"],
    "Oceania": ["Australia and New Zealand", "Micronesia", "Melanesia", "Polynesia"],
    "Antarctica": ["Antarctica"]
}

# Function to determine region and subregion
def get_region_subregion(country):
    for subregion, countries in un_geoscheme.items():
        if country in countries:
            for region, subregions in region_mapping.items():
                if subregion in subregions:
                    return region, subregion
    return None, None

# Add region and subregion columns
countries_df['region'], countries_df['subregion'] = zip(*countries_df['name'].map(get_region_subregion))

In [54]:
# Create the region and subregion colour mappings based on the provided image data
region_colour_mapping = {
    'Africa': '#3e05ee',
    'Americas': '#194ce2',
    'Antarctica': '#444444',
    'Asia': '#196c3d',
    'Europe': '#e5b926',
    'Oceania': '#dc0000'
}

subregion_colour_mapping = {
    'Middle Africa': '#7733b3',
    'Western Africa': '#9c73d6',
    'Eastern Africa': '#ab18e2',
    'Southern Africa': '#d995de',
    'Northern Africa': '#ae99e7',
    'Caribbean': '#4b76e2',
    'South America': '#99e0e7',
    'Northern America': '#3392b3',
    'Central America': '#7bc6e0',
    'Antarctica': '#444444',
    'Western Asia': '#bbe4ac',
    'Southern Asia': '#5d9c41',
    'Southeastern Asia': '#b6e24b',
    'Eastern Asia': '#7b965e',
    'Central Asia': '#73d694',
    'Southern Europe': '#ffe599',
    'Western Europe': '#d2a740',
    'Eastern Europe': '#fbe426',
    'Northern Europe': '#f6b26b',
    'European Union': '#ce7e00',
    'Polynesia': '#f78181',
    'Australia and New Zealand': '#cc0000',
    'Melanesia': '#e93f7b',
    'Micronesia': '#9b1f1f'
}

# Add the regionColour and subregionColour columns to the countries dataframe
countries_df['regionColour'] = countries_df['region'].map(region_colour_mapping)
countries_df['subregionColour'] = countries_df['subregion'].map(subregion_colour_mapping)

In [62]:
post_counts.to_csv('post_counts.csv', index=False)
df_combined.to_csv('network_power.csv', index=False)
node_attributes.to_csv('country_attributes.csv', index=False)
countries_df.to_csv('countries.csv', index=False)